In [13]:
API_KEY=''

import requests
import json
import time
import os

class GooglePlaces(object):
    def __init__(self, apiKey):
        super(GooglePlaces, self).__init__()
        self.apiKey = apiKey
    
    def search_places_by_text(self, text):
        endpoint_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        params = {
            'input': text,
            'inputtype': 'textquery',
            'fields' : 'name,rating,user_ratings_total,place_id,geometry/location',
            'key': self.apiKey
        }
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        print(results)
        return results
    
plc = GooglePlaces(API_KEY)

iata_codes = [
    "BHM","DHN","HSV","MOB","MGM","ANC","FAI","JNU","FLG","PHX","TUS",
    "YUM","FYV","LIT","XNA","BUR","FAT","LGB","LAX","OAK","ONT","PSP",
    "SMF","SAN","SFO","SJC","SNA","ASE","COS","DEN","GJT","PUB","BDL",
    "HVN","IAD","DCA","DAB","FLL","RSW","JAX","EYW","MIA","MCO","PNS",
    "PIE","SRQ","TPA","PBI","PFN","ATL","AGS","SAV","ITO","HNL","OGG",
    "KOA","LIH","BOI","MDW","ORD","MLI","PIA","EVV","FWA","IND","SBN",
    "CID","DSM","ICT","LEX","SDF","BTR","MSY","SHV","AUG","BGR","PWM",
    "BWI","BOS","HYA","ACK","ORH","BTL","DTW","DET","FNT","GRR","AZO",
    "LAN","MBS","DLH","MSP","RST","GPT","JAN","MCI","STL","SGF","BIL",
    "LNK","OMA","LAS","RNO","MHT","ACY","EWR","TTN","ABQ","ALM","ALB",
    "BUF","ISP","JFK","LGA","SWF","ROC","SYR","HPN","AVL","CLT","FAY",
    "GSO","RDU","INT","BIS","FAR","CAK","CVG","CLE","CMH","DAY","TOL",
    "OKC","TUL","EUG","PDX","HIO","SLE","ABE","ERI","MDT","PHL","PIT",
    "AVP","PVD","CHS","CAE","GSP","MYR","PIR","RAP","FSD","TRI","CHA",
    "TYS","MEM","BNA","AMA","AUS","CRP","DAL","DFW","ELP","HOU","IAH",
    "LBB","MAF","SAT","SLC","BTV","MPV","RUT","IAD","PHF","ORF","RIC",
    "ROA","PSC","SEA","GEG","CRW","CKB","HTS","GRB","MSN","MKE","CPR",
    "CYS","JAC","RKS"]

FNAME = "places.json"
if os.path.exists(FNAME):
    with open(FNAME, 'r') as infile:
        places = json.load(infile)
else:
    places = []
    for iata_code in iata_codes:
        r = plc.search_places_by_text("{} airport".format(iata_code))
        places.extend([c for c in r['candidates'] if "rating" in c])

In [14]:
%matplotlib inline
import plotly 
import plotly.graph_objects as go
import pandas as pd

df = pd.DataFrame(places)
df = df[df["user_ratings_total"] > 500]
df["lat"] = df.apply(lambda r: r["geometry"]["location"]["lat"], axis=1)
df["lng"] = df.apply(lambda r: r["geometry"]["location"]["lng"], axis=1)
df["text"] = df["name"] + " " + df["rating"].astype(str)

### Worst airports

In [8]:
print(df[["name", "rating"]].sort_values("rating").head(5))

                                        name  rating
116                        LaGuardia Airport     2.7
108     Newark Liberty International Airport     3.0
98         Kansas City International Airport     3.0
144       Philadelphia International Airport     3.3
122  Charlotte Douglas International Airport     3.3


### Best airports

In [9]:
print(df[["name", "rating"]].sort_values("rating", ascending=False).head(5))

                                   name  rating
46          Tampa International Airport     4.4
138      Portland International Airport     4.4
15            Hollywood Burbank Airport     4.4
17                   Long Beach Airport     4.4
65   Indianapolis International Airport     4.4


In [15]:
fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df['lng'],
    lat = df['lat'],
    text = df['text'],
    marker = dict(
        size = df['user_ratings_total'].clip(lower=1000.)/50,
        
        colorscale = 'rdylgn',
        cmin = 3.0,
        color = df['rating'],
        cmax = 4.5,
        
        line_color='rgb(40,40,40)',
        line_width=0.5,
        sizemode = 'area'
    ),
    name = "Airport ratings"))

fig.update_layout(
    width=1800,
    height=1200,
    title_text = "Airport Ratings",
    showlegend = True,
    geo = dict(
        scope = 'usa',
        landcolor = 'rgb(217, 217, 217)',
    )
)

fig.show()